In [138]:
# Configurações Google Drive
import os
from google.colab import drive
drive.mount('/content/drive')
workdir_path = '/content/drive/MyDrive/ag_acoes'
os.chdir(workdir_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [139]:
# Imports
import pandas as pd
import datetime

In [140]:
# Importa o dicionário de dadoslayout de dados do arquivo de cotações
import shutil
shutil.copy(os.path.join(workdir_path, 'conf.py'), '/content')
import conf

In [141]:
# Definições
DIR_FILES = os.path.join(workdir_path, 'Cotações')
N_TOP_VOLUME_ANO = 300 # Filtar a quantidade de ações com maiores volumes ano a ano

In [142]:
# Lista os arquivos a serem processados
def lista_arquivos_processar():
    """
    Lista os arquivos a serem processados
    """
    # Lista os arquivos a serem importados
    files = []
    for r, d, f in os.walk(DIR_FILES):
        for file in f:
            if '.txt' in file.lower():
                files.append(os.path.join(r, file))
                
    # Retorno
    # files = files[:1] # Só para teste
    return sorted(files)

In [143]:
def transforma_arq_df(file_path):
    """
    Recebe o arquivo de cotações e retorna um dataframe
    """    
    with open(file_path, 'r') as f:
        content = f.readlines()
    f.close()
    
    seg_content = []
    for reg in content:
        seg_reg = []
        
        for item in conf.DIC_META_DADO_BDI_HIST:
            ini = conf.DIC_META_DADO_BDI_HIST[item]['pos_inicial']
            fim = conf.DIC_META_DADO_BDI_HIST[item]['pos_final']
            tipo = conf.DIC_META_DADO_BDI_HIST[item]['tipo']
            
            value = reg[ini-1:fim]
            
            try:
                if tipo == str:
                    value = value.strip()
                elif tipo == datetime.date:
                    ano = int(value[:4])
                    mes = int(value[4:6])
                    dia = int(value[6:])
                    
                    value = datetime.date(ano, mes, dia)
                elif tipo == float:
                    value = float(value) / 100
                elif tipo == int:
                    value = value.strip()
                    if value == '':
                        value = 0
                    else:
                        value = int(value)
            except:
                # print("%s => %s" %(reg, value))
                pass
                
            seg_reg.append(value)
        
        if seg_reg[0] == 1:    
            seg_content.append(seg_reg)
            
    # Transforma os dados para pandas    
    df_cotacoes_hist = pd.DataFrame(data=seg_content, columns=(list(conf.DIC_META_DADO_BDI_HIST)))
    
    # Filtra tudo que é negociação de AÇÃO
    df_cotacoes_hist = df_cotacoes_hist.loc[(df_cotacoes_hist['TPMERC'] == 10) &
                                            (df_cotacoes_hist['CODBDI'] == 2)].copy()
    
    # Retorno
    return df_cotacoes_hist


In [144]:
# Importa os dados dos arquivos de cotações e insera as informações de variação futura
df_cotacoes = consolida_cotacoes()
df_cotacoes.head()

Processando /content/drive/MyDrive/ag_acoes/Cotações/COTAHIST_A2018.TXT
Processando /content/drive/MyDrive/ag_acoes/Cotações/COTAHIST_A2019.TXT
Processando /content/drive/MyDrive/ag_acoes/Cotações/COTAHIST_A2020.TXT
Processando /content/drive/MyDrive/ag_acoes/Cotações/COTAHIST_A2021.TXT


,DATA_PREGAO,CODNEG,PREULT,VOLTOT
26358,2020-02-07,A1AP34,143.12,6139848.0
26359,2020-02-10,A1AP34,142.27,512172.0
26360,2020-02-13,A1AP34,147.37,235792.0
26361,2020-02-18,A1AP34,155.53,454900.3
35946,2020-02-19,A1AP34,153.42,154954.2


In [145]:
# Verifica ano a ano as ações que tiveram os maiores volumes de negociação,
# limitados em N_TOP_VOLUME_ANO

# Insere o ano
df_cotacoes['ANO'] = df_cotacoes['DATA_PREGAO'].apply(lambda x: x.year)

# Inicializa a lista de ações
list_acoes = []

# Exibe a quantidade total de ações
print("Quantidade total de ações: {}".format(df_cotacoes['CODNEG'].nunique()))

# Para cada ano, verifica as ações com maiores negociações
for ano in sorted(df_cotacoes['ANO'].unique()):
  list_acoes_ano = df_cotacoes.loc[df_cotacoes['ANO']==ano].groupby(['CODNEG']).sum()['VOLTOT'].nlargest(N_TOP_VOLUME_ANO).index.tolist()
  if list_acoes == []: list_acoes = list_acoes_ano
  [list_acoes.remove(acao) for acao in list_acoes if acao not in list_acoes_ano]

  print("Ano: {}. Ações: {}. Quantidade acumulada de ações: {}".format(ano, len(list_acoes_ano), len(list_acoes)))

Quantidade total de ações: 1290
Ano: 2018. Ações: 300. Quantidade acumulada de ações: 300
Ano: 2019. Ações: 300. Quantidade acumulada de ações: 267
Ano: 2020. Ações: 300. Quantidade acumulada de ações: 233
Ano: 2021. Ações: 300. Quantidade acumulada de ações: 206


In [146]:
# Filtra somente as ações selecionadas
df_cotacoes = df_cotacoes.loc[df_cotacoes['CODNEG'].isin(list_acoes)].copy()
df_cotacoes.head()

,DATA_PREGAO,CODNEG,PREULT,VOLTOT,ANO
0,2018-01-02,AALR3,14.89,1402991.0,2018
2058,2018-01-03,AALR3,14.96,8765964.0,2018
4255,2018-01-04,AALR3,15.09,4355491.0,2018
6632,2018-01-05,AALR3,15.05,1982292.0,2018
8954,2018-01-08,AALR3,14.79,3620453.0,2018


In [147]:
!pip install yahooquery
from yahooquery import Ticker

In [ ]:
df_yahoo = pd.DataFrame()
dic_errors = {}
list_acoes.sort()
for acao in list_acoes:
  acao_sufix = acao + '.SA'
  # print(acao)
  try:
    historico = Ticker(acao_sufix)
    df = historico.history(start='2015-01-01', end='2021-01-01')
    df = df.reset_index()[['symbol', 'date', 'adjclose']].copy()
    df['symbol'] = acao
    df_yahoo = pd.concat([df_yahoo, df])
  except Exception as exc:
    dic_errors[acao_sufix] = str(exc)
    print("   Erro {}".format(acao))

   Erro GBIO33
   Erro IBOV11
   Erro IDNT3


In [ ]:
# Adiciona o índice Bovespa
historico = Ticker('^BVSP')
df = historico.history(start='2015-01-01', end='2021-01-01')
df = df.reset_index()[['symbol', 'date', 'adjclose']].copy()
df_yahoo = pd.concat([df_yahoo, df])

In [ ]:
df_yahoo.symbol.nunique()

204

In [ ]:
df_yahoo.to_excel('df_yahoo_finance.xlsx')

In [ ]:
df = historico.history(start='2015-01-01', end='2021-01-01')

In [ ]:
df

low      open      high     close     volume  adjclose
symbol date                                                                   
^BVSP  2015-01-02   48345.0   50005.0   50005.0   48512.0  2882100.0   48512.0
       2015-01-05   47264.0   48512.0   48512.0   47517.0  3866100.0   47517.0
       2015-01-06   47338.0   47517.0   48061.0   48001.0  4559300.0   48001.0
       2015-01-07   48006.0   48006.0   49882.0   49463.0  4408800.0   49463.0
       2015-01-08   49017.0   49463.0   50261.0   49943.0  3621900.0   49943.0
...                     ...       ...       ...       ...        ...       ...
       2020-12-22  115648.0  115825.0  116903.0  116348.0  6947900.0  116348.0
       2020-12-23  116636.0  116636.0  118311.0  117857.0  6483300.0  117857.0
       2020-12-28  117805.0  117806.0  119213.0  119051.0  7171700.0  119051.0
       2020-12-29  118750.0  119130.0  119861.0  119475.0  6769700.0  119475.0
       2020-12-30  118919.0  119410.0  120150.0  119306.0  8235700.0  119306.0

[1482 rows x 6 columns]